In [1]:
# import dependencies
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

warnings.filterwarnings("ignore")

### Pre-processing

In [2]:
df_107 = pd.read_csv(("../Resources/107justice.csv"))
df_107.tail()

,justice,term,direction,precedentAlteration,issueArea,lawType,caseOriginState
1685,107,2008,2,0,1,2,0
1686,107,2008,2,0,8,3,0
1687,107,2008,2,0,2,4,0
1688,107,2008,1,0,10,6,0
1689,107,2008,2,0,2,3,0


In [3]:
# df_107["direction"] = np.where(df_106["direction"] == 1, 0, 1)
# df_107.tail(5)
# # df_107.count()

In [4]:
target_107 = df_107["direction"]
target_names = ["conservative", "liberal"]

In [5]:
data_107 = df_107.drop(["direction", "justice"], axis=1)
feature_names = data_107.columns
data_107.head()

,term,precedentAlteration,issueArea,lawType,caseOriginState
0,1990,0,9,4,0
1,1990,0,10,0,0
2,1990,0,10,0,0
3,1990,0,1,2,22
4,1990,0,1,2,0


In [6]:
X_scaler = StandardScaler().fit(data_107)
X_scaled = X_scaler.transform(data_107)

X = X_scaled
y = target_107

### Evaluating different classifiers

In [7]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1, n_estimators=300)
clf3 = GaussianNB()
clf4 = SVC(kernel='linear', probability=True)

print('5-fold cross validation:\n')

labels = ["Logistic Regression", "Random Forest", "Naive Bayes", "Linear SVM"]

for clf, label in zip([clf1, clf2, clf3, clf4], labels):
    scores = cross_val_score(clf, X, y, cv=5, 
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))


5-fold cross validation:

F1: 0.141 [Logistic Regression]
F1: 0.277 [Random Forest]
F1: 0.247 [Naive Bayes]
F1: 0.000 [Linear SVM]


In [8]:
# take out low-scoring classifiers as voting classifier estimators
v_clf_hard = VotingClassifier(estimators = [(labels[2], clf3),
                                            (labels[3], clf4)],
                                            voting = 'hard')

In [9]:
v_clf_soft = VotingClassifier(estimators = [(labels[2], clf3),
                                            (labels[3], clf4)],
                                            voting = 'soft')

In [10]:
new_labels = ["Random Forest", "Naive Bayes", 
              "Voting_Classifier_Hard", "Voting_Classifier_Soft"]

for (clf, label) in zip([clf2, clf3, v_clf_hard, v_clf_soft], new_labels):
    scores = cross_val_score(clf, X, y,cv=5,
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

F1: 0.277 [Random Forest]
F1: 0.247 [Naive Bayes]
F1: 0.247 [Voting_Classifier_Hard]
F1: 0.122 [Voting_Classifier_Soft]


## F1 Scores are too low

### Naive Bayes Model Training Below as example

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [13]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.555
Test Acc: 0.496


In [14]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.40      0.60      0.48       163
     liberal       0.63      0.43      0.51       260

   micro avg       0.50      0.50      0.50       423
   macro avg       0.51      0.51      0.50       423
weighted avg       0.54      0.50      0.50       423



In [15]:
confusion_matrix(predictions, y_test)
predictions[:10], y_test[:10].ravel()

(array([2, 2, 2, 2, 1, 1, 2, 2, 2, 2], dtype=int64),
 array([1, 2, 2, 1, 2, 1, 2, 2, 1, 1], dtype=int64))